# chinese graph

In [66]:
# coding: utf-8

import codecs
import networkx as nx
import numpy as np

In [128]:
f = codecs.open('./chinesegraph/chinese_words.csv', 'rb', encoding='utf-8')
words = [line.strip() for line in f]\
print 'number of words : ', len(words)

In [130]:
chars = set()
for w in words:
    chars = chars.union(set( w ))
    # Creating a little 'index'
print 'number of chars : ', len(chars)        

In [134]:
G = nx.Graph()
for ch in chars:
    G.add_node(ch,{'chinese':ch})

print 'number of chars nodes : ', G.number_of_nodes()

In [137]:
for i,w in enumerate(words):
    G.add_node(str(i),{'chinese': w })
    for ch in w:
        G.add_edge(str(i),ch)

print 'number of nodes (chars + words): ', G.number_of_nodes()

nx.write_graphml(G,'chinese_graph.graphml')

# word embedding

In [190]:
import networkx as nx
import numpy as np
import scipy as sp
import time
from random import choice
from math import log
import codecs

In [320]:
IS_CHINESE_TEST = True

if IS_CHINESE_TEST:
    G = nx.read_graphml('./data/graphs/chinese_graph.graphml', unicode) # Chinese graph
else:
    G = nx.read_graphml('graphs/bipartite_graph.graphml', unicode) # Korean language graph    

In [321]:
gen = nx.connected_component_subgraphs(G)
G = next(gen)
output_dir = "tmpDistancesFull2"

In [322]:
sts = nx.bipartite.sets(G)

In [323]:
words = None
hanjas = None
wordSet = None

if len(sts[0]) > 6000: 
    words = list(sts[0])
    wordSet = sts[0]
    hanjas = list(sts[1])
else: 
    words = list(sts[1])
    wordSet = sts[1]
    hanjas = list(sts[0])


In [324]:
wordic = {G.node[e]['chinese']:k for k,e in enumerate(words)}

In [325]:
synonyms = None

In [326]:
with codecs.open('./data/synonyms/all_chinese_synonyms.csv', 'r', encoding='utf-8') as f:
        synonyms = [line.strip() for line in f]

In [327]:
synDP = []
randDP = []


In [328]:
A = nx.bipartite.biadjacency_matrix(G, hanjas)
A = A.astype(float)

In [329]:
weight = False # In case we chose not to do TF-IDF weighting

In [330]:
minK = 666
maxK = 668

In [331]:
U,s,V = sp.sparse.linalg.svds(A,800)
Vtr = V.transpose()

In [316]:
for pr in synonyms:

    word = pr.split(',')
    c1 = word[0]
    c2 = word[1]
    
    if c1 not in wordic or c2 not in wordic: continue

    v1 = Vtr[wordic[c1]]
    v2 = Vtr[wordic[c2]]
    synDP.append([ np.dot(v1,v2), c1.encode('utf-8'), c2.encode('utf-8') ] )

In [317]:
for _ in synonyms:
    v1 = choice(Vtr)
    v2 = choice(Vtr)
    randDP.append(np.dot(v1,v2))

In [318]:
output_dir = './output'
with codecs.open("{}/randdist_k={}.csv".format(output_dir,k), 'w', encoding='utf-8') as f:
    wr = csv.writer(f)
    wr.writerows([[r] for r in randDP])

In [319]:
with codecs.open("{}/syndist_k={}.csv".format(output_dir,k), 'w') as f:
    wr = csv.writer(f)
    wr.writerows(synDP)

# scrap chinese tools

In [21]:
from bs4 import BeautifulSoup
import urllib
import networkx as nx

In [22]:
G = nx.read_graphml('data/graphs/chinese_graph.graphml', unicode)

In [23]:
words = list(G.node[a]['chinese'] for a in G.node.keys() if 'chinese' in G.node[a])

In [4]:
base_url = "https://www.chinesetools.eu/tools/synonym/"

In [24]:
len(words)

122689

In [6]:
def url_encoding(data) :
    tmp = ''
    for i in xrange( len(data) ):
        tmp = tmp + '&#' + str(ord(data[i])) + ';'
    
    return urllib.quote(tmp)

In [3]:
# for i, ch in enumerate(words):
#     if len(ch) is 2:
#         print i, ch
# 46

w = words[4]
print w

若无其事


In [4]:
syn_pairs = []
query = base_url + '?q=' + url_encoding(w) + '&Submit=Search'
r = urllib.urlopen(query)
soup = BeautifulSoup(r)

parse = soup.find_all('div', 'arrondi_10')

# if ( len(parse) < 3 ):

# find the 'divs' which contain the synonym 
divs = parse[1].find_all('div')

inner_finder = 0

for i, dv in enumerate(divs):
    if dv.string == 'Click on the synonyms to see it on the Chinese dictionary:' :
        inner_finder = i
        
for dv in divs[inner_finder + 1].find_all('div'):
    for a in dv.find_all('a'):
        print a.string
        syn_pairs.append([w,a.string])

不动声色
处之泰然
处变不惊
守静
定神
宠辱不惊
措置裕如
毫不动摇
沉住气
沉着
波澜不惊
泰然处之
泰然自若
满不在乎
熙和恬静
稳如泰山
行若无事
见惯不惊
谈笑自若
镇定
镇定自若
镇静
面不改色


In [59]:
with codecs.open('chinesetools_synonym.csv','w', encoding='utf-8') as f:
    for item in syn_pairs:
        print item[0]
        f.write("%s\n" % item[0])
    #wr = csv.writer(f)
    #wr.writerows(syn_pairs.encode('u'))

In [1]:
from bs4 import BeautifulSoup
import urllib
import networkx as nx
import codecs

G = nx.read_graphml('data/graphs/chinese_graph.graphml', unicode)
words = list(G.node[a]['chinese'] for a in G.node.keys() if 'chinese' in G.node[a])

base_url = "https://www.chinesetools.eu/tools/synonym/"

def url_encoding(data) :
    tmp = ''
    for i in xrange( len(data) ):
        tmp = tmp + '&#' + str(ord(data[i])) + ';'
    
    return urllib.quote(tmp)


In [16]:

f = codecs.open('chinesetools_synonym.csv','w', encoding='utf-8')

syn_pairs = []

for i,w in enumerate(words):

    print("[{}/{}] now processing: {} , got {} synonyms"
              .format(i,len(words), w.encode('utf-8'), len(syn_pairs)))        
    
    query = base_url + '?q=' + url_encoding(w) + '&Submit=Search'
    r = urllib.urlopen(query)
    soup = BeautifulSoup(r)

    parse = soup.find_all('div', 'arrondi_10')

    if ( len(parse) < 2 ):
        continue

    # find the 'divs' which contain the synonym 
    divs = parse[1].find_all('div')

    inner_finder = -1

    for i, dv in enumerate(divs):
        if dv.string == 'Click on the synonyms to see it on the Chinese dictionary:' :
            inner_finder = i

    if inner_finder is -1:
        continue
            
    for dv in divs[inner_finder + 1].find_all('div'):
        for a in dv.find_all('a'):
            print w, a.string.strip()
            syn_pairs.append([w, a.string.strip()])
            f.write("%s\n" % (w + ',' + a.string.strip()))

[0/122689] now processing: 萨里奇 , got 0 synonyms
[1/122689] now processing: 布罗德里克 , got 0 synonyms
[2/122689] now processing: 若干年 , got 0 synonyms
[3/122689] now processing: 若干年内 , got 0 synonyms
[4/122689] now processing: 若无其事 , got 0 synonyms
若无其事 不动声色
若无其事 处之泰然
若无其事 处变不惊
若无其事 守静
若无其事 定神
若无其事 宠辱不惊
若无其事 措置裕如
若无其事 毫不动摇
若无其事 沉住气
若无其事 沉着
若无其事 波澜不惊
若无其事 泰然处之
若无其事 泰然自若
若无其事 满不在乎
若无其事 熙和恬静
若无其事 稳如泰山
若无其事 行若无事
若无其事 见惯不惊
若无其事 谈笑自若
若无其事 镇定
若无其事 镇定自若
若无其事 镇静
若无其事 面不改色
[5/122689] now processing: 若有证据 , got 23 synonyms
[6/122689] now processing: 若泽 , got 23 synonyms
[7/122689] now processing: 若羌县 , got 23 synonyms
[8/122689] now processing: 若许 , got 23 synonyms
[9/122689] now processing: 若隐若现 , got 23 synonyms
若隐若现 不明
若隐若现 依稀
若隐若现 幽渺
若隐若现 影影绰绰
若隐若现 微茫
若隐若现 恍
若隐若现 恍恍忽忽
若隐若现 恍惚
若隐若现 惺忪
若隐若现 朦朦
若隐若现 朦胧
若隐若现 模模糊糊
若隐若现 模糊
若隐若现 模糊不清
若隐若现 渺无音信
若隐若现 渺茫
若隐若现 白蒙蒙
若隐若现 盲用
若隐若现 盲目
若隐若现 糊涂
若隐若现 糊里糊涂
若隐若现 缥缈
若隐若现 胡里胡涂
若隐若现 若明若暗
若隐若现 莫明其妙
若隐若现 莽苍
若隐若现 蒙胧
若隐若现 迷茫
若隐若现 迷蒙
若隐若现 隐约
若隐若现 隐约可见
若隐若现 隐隐
若隐若现 隐隐约约
若隐若现 雾

KeyboardInterrupt: 

In [ ]:
with codecs.open('chinesetools_synonym.csv','w', encoding='utf-8') as f:
    for item in syn_pairs:
        print item[0]
        f.write("%s\n" % item[0])

In [7]:
print w, a.string.strip()

若无其事 面不改色


In [11]:
syn_pairs.append([w, a.string.strip()])

In [12]:
kk = w + a.sring.strip()

AttributeError: 'NoneType' object has no attribute 'strip'

In [18]:
inner_finder = -1

In [1]:

import networkx as nx
import numpy as np
import scipy as sp
import time
from random import choice
from math import log
import codecs
import csv
import dato_utils


In [2]:
IS_CHINESE_TEST = True

if IS_CHINESE_TEST:
    G = nx.read_graphml('./data/graphs/chinese_graph.graphml', unicode) # Chinese graph
else:
    G = nx.read_graphml('graphs/bipartite_graph.graphml', unicode) # Korean language graph    

In [3]:
gen = nx.connected_component_subgraphs(G)
G = next(gen)

In [4]:
dato_utils.create_dir('result')
output_dir = "./result"

In [5]:
sts = nx.bipartite.sets(G)

In [21]:
len(sts[0])

117116

In [6]:
words = None
hanjas = None
wordSet = None
if len(sts[0]) > 6000: 
    words = list(sts[0])
    wordSet = sts[0]
    hanjas = list(sts[1])   # root
else: 
    words = list(sts[1])
    wordSet = sts[1]
    hanjas = list(sts[0])

In [7]:
wordic = {G.node[e]['chinese']:k for k,e in enumerate(words)}

In [8]:
synonyms = None
# synonyms delimeter = ',' 
if IS_CHINESE_TEST:
    with codecs.open('./data/synonyms/chinesetools_synonyms.csv', 'r', encoding='utf-8') as f:
        synonyms = [line.strip() for line in f]
        
else:
    with codecs.open('./data/synonums/korean_synonyms.csv', 'r', encoding='utf-8') as f:
        synonyms = [line.strip() for line in f]

In [9]:
print synonyms[3]

妪,老太婆


In [10]:
synDP = []
randDP = []

A = nx.bipartite.biadjacency_matrix(G, hanjas)
#A = A.asfptype()
#A = A.tolil()
A = A.astype(float)
weight = False # In case we chose not to do TF-IDF weighting

In [29]:
minK = 100
maxK = 110

In [20]:
k = 500
print("Factorizing for k={}".format(k))
U,s,V = sp.sparse.linalg.svds(A, k)
Vtr = V.transpose()

Factorizing for k=500


In [33]:
for _ in synonyms:
    print _
    break

赃,贼赃


In [21]:
aa = synonyms[10]

In [22]:
c = aa.split(',')

In [24]:
v1 = Vtr[wordic[c[0]]]
v2 = Vtr[wordic[c[1]]]

In [28]:
from sklearn.metrics.pairwise import cosine_similarity

In [40]:
cosine_similarity(v1, v2)[0][0]

/usr/local/lib/python2.7/dist-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


-0.002946969290243294

In [41]:
np.dot(v1, v2)

-1.0592178303158552e-05

In [18]:
import csv
import matplotlib.pyplot as plt
import seaborn as sns
from statistics import stdev
import argparse


In [19]:
synDist = None
ranDist = None

dim = 400

missRand = []
gotSyn = []
ks = []

homDir = './result'

with open('{}/randdist_k={}.csv'.format(homDir, dim)) as f:
    ranDist = list(csv.reader(f))
    ranDist = [float(a[0]) for a in ranDist]

with open('{}/syndist_k={}.csv'.format(homDir, dim)) as f:
    synDist = list(csv.reader(f))
    synDist = [float(a[0]) for a in synDist]

IOError: [Errno 2] No such file or directory: './result/randdist_k=400.csv'

In [9]:
sd = stdev(ranDist)
ks.append(dim)
missRand.append(sum(1.0 if abs(rd) > sd else 0.0 for rd in ranDist)/len(ranDist))
gotSyn.append(sum(1.0 if abs(rd) > sd else 0.0 for rd in synDist)/len(synDist))

In [14]:
missRand

[0.027351117251228237]

In [15]:
gotSyn

[0.03417677542630497]

In [47]:
G = nx.read_graphml('./data/graphs/chinese_graph.graphml', unicode) # Chinese graph

In [48]:
sts = nx.bipartite.sets(G)

In [65]:
a = []

In [69]:
a = list(sts[0]) + list(sts[1])

In [71]:
with codecs.open('./data/synonyms/chinesetools_synonyms.csv', 'r', encoding='utf-8') as f:
        synonyms = [line.strip() for line in f]

In [74]:
synonyms[0]

u'\u8d43,\u8d3c\u8d43'